# Proving the Playpen Agent/CodePlan for Compiler Errors

We need prove that the playpen code plan agents workflow will work as expected. This will be a way to run them and make sure that things work as we expect.

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2
# Install local kai package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install -e ../../

sys.path.append("../../")

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=2)

## Create the Test Environment

For creating the test environment we will:

1. move the coolstore to a temp direcotry
2. Update the InventoryEntity to use UUID for the itemID as the type.

In [ ]:
import tempfile
import shutil
import os
import subprocess

#First we need to download coolstore if it does not exist.

if not os.path.exists("./coolstore"):
    subprocess.run("../../example/fetch.sh")

temp_dir = tempfile.TemporaryDirectory()
# coolstore_path = os.path.join(temp_dir.name, "coolstore")
coolstore_path = "/home/fabian/projects/github.com/konveyor-ecosystem/kai/example/coolstore"
# shutil.copytree("./coolstore", coolstore_path)

## make the necessary change
shutil.copyfile("./testing_field_type_change_errors/InventoryEntity.java", f"{coolstore_path}/src/main/java/com/redhat/coolstore/model/InventoryEntity.java")
shutil.copyfile("./testing_field_type_change_errors/CatalogItemEntity.java", f"{coolstore_path}/src/main/java/com/redhat/coolstore/model/CatalogItemEntity.java")

print(temp_dir)


## Set up the Code Plan types and run code plan.

In [ ]:
from kai.repo_level_awareness.api import Agent, RpcClientConfig, Task, TaskResult, ValidationStep
from kai.repo_level_awareness.codeplan import TaskManager
from kai.repo_level_awareness.agents.compiler_agent import MavenCompilerAgent
from kai.repo_level_awareness.maven_validator import MavenCompileStep
from kai.service.llm_interfacing.model_provider import ModelProvider
from kai.models.kai_config import KaiConfig


config = RpcClientConfig(coolstore_path)

kai_config = KaiConfig.model_validate_filepath("../../kai/config.toml")
modelProvider = ModelProvider(kai_config.models)


agent = MavenCompilerAgent(modelProvider.llm)

task_manager = TaskManager(
        config,
        None,
        validators=[MavenCompileStep(config)],
        agents=[agent],
    )


for task in task_manager.get_next_task():
    pp.pprint(task)
    task_manager.supply_result(task_manager.execute_task(task))

## Cleanup temp dir

In [ ]:
temp_dir.cleanup()